### SETUP INICIAL DO PROJETO

In [1]:

#importação das bibliotecase e pacotes necessários para a análise

import json
import numpy as np
import os
import pandas as pd
import pandas_gbq as gbq
import re
import seaborn as sns
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from google.cloud import bigquery
from google.cloud.bigquery_storage import BigQueryReadClient
from google.oauth2 import service_account


# Carrega o .env: onde estão as credenciais do projeto/repositório
load_dotenv("/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/.env")

# Detectar ambiente: como eu estou usando wsl-ubuntu, no VS Code  -  Windows, estava dando conflitos de path
if os.name == "nt":  # se Windows
    credentials_path = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"
else:  # se WSL/Linux
    credentials_path = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"

# Parâmetros injetados pelo Papermill ou definidos manualmente, caso não existam no ambiente
# Tables_to_process: lista de tabelas que serão processadas
# Output_dataset: nome do dataset onde os dados processados serão armazenados, neste caso, raw_data_cleaned
if 'tables_to_process' not in locals():
    tables_to_process = [
        "desafioadventureworks-446600.stg_marts_tables.fact_product_forecast"  ,
        "desafioadventureworks-446600.stg_marts_tables.fact_product_seasonality",
        "desafioadventureworks-446600.stg_staging_tables.stg_production_product",
        "desafioadventureworks-446600.stg_staging_tables.stg_sales_store"

    ]

if 'output_dataset' not in locals():
    output_dataset = "desafioadventureworks-446600.stg_marts_tables"

# Configs do cliente BigQuery: input de project e location de acordo com dados no Bigquery
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=os.getenv("BIGQUERY_PROJECT"), location="us-central1")


In [2]:
# Print com a tabela que vai ser processada nesse notebook

print("Tabelas a processar:", tables_to_process)

Tabelas a processar: ['desafioadventureworks-446600.stg_marts_tables.fact_product_forecast', 'desafioadventureworks-446600.stg_marts_tables.fact_product_seasonality', 'desafioadventureworks-446600.stg_staging_tables.stg_production_product', 'desafioadventureworks-446600.stg_staging_tables.stg_sales_store']


# Exploratory Data Analysis (EDA) e Data Cleaning

### Glossário dos dados:

O termo ''doc:'', situado no rodapé de algumas cells, indica algo como:

- documentação: documentar decisões, análises e resultados;

- abreviações de termos, como bkp, df, entre outros.

In [3]:
# Setup inicial do df 

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 10000)
pd.options.display.float_format = '{:.2f}'.format


#doc: df = dataframe  

In [4]:
# Dicionário para armazenar os df processados
df_processados = {}

# Iteração das tabelas e armazenamento em df
for input_table in tables_to_process:
    print(f"Processando tabela: {input_table}")
    
    table_name = input_table.split(".")[-1].replace("-", "_")  
    
    print("Lendo os dados do BigQuery...")
    query = f"SELECT * FROM `{input_table}`"
    table_data = client.query(query).to_dataframe()
    
    df_processados[table_name] = table_data
    print(f"Tabela {table_name} processada e armazenada com sucesso.")


print("Todas as tabelas foram processadas com sucesso!")

Processando tabela: desafioadventureworks-446600.stg_marts_tables.fact_product_forecast
Lendo os dados do BigQuery...
Tabela fact_product_forecast processada e armazenada com sucesso.
Processando tabela: desafioadventureworks-446600.stg_marts_tables.fact_product_seasonality
Lendo os dados do BigQuery...
Tabela fact_product_seasonality processada e armazenada com sucesso.
Processando tabela: desafioadventureworks-446600.stg_staging_tables.stg_production_product
Lendo os dados do BigQuery...
Tabela stg_production_product processada e armazenada com sucesso.
Processando tabela: desafioadventureworks-446600.stg_staging_tables.stg_sales_store
Lendo os dados do BigQuery...
Tabela stg_sales_store processada e armazenada com sucesso.
Todas as tabelas foram processadas com sucesso!


In [5]:
# Listar todas as variáveis criadas dinamicamente
for table_name in df_processados.keys():
    print(f"Variável criada: {table_name}")  

Variável criada: fact_product_forecast
Variável criada: fact_product_seasonality
Variável criada: stg_production_product
Variável criada: stg_sales_store


In [6]:
# Atribuir o df a uma variável com nome mais simples
fact_product_forecast = df_processados['fact_product_forecast']

print(f"Colunas: {fact_product_forecast.shape[1]}\nLinhas: {fact_product_forecast.shape[0]}")

Colunas: 6
Linhas: 106840


# Questão 11

In [7]:
# Consolidando os dados para previsão
fact_product_seasonality = df_processados['fact_product_seasonality']
fact_product_forecast = df_processados['fact_product_forecast']
stg_production_product = df_processados['stg_production_product']

In [8]:
# Listar todas as variáveis definidas no ambiente atual (corrigido)
variables_snapshot = list(globals().items())

for var_name, var_value in variables_snapshot:
    if isinstance(var_value, pd.DataFrame):
        print(f"DataFrame: {var_name} - Dimensão: {var_value.shape}")


DataFrame: table_data - Dimensão: (701, 5)
DataFrame: fact_product_forecast - Dimensão: (106840, 6)
DataFrame: fact_product_seasonality - Dimensão: (35, 5)
DataFrame: stg_production_product - Dimensão: (504, 24)


In [9]:
print("\nVisualizando os primeiros registros de stg_production_product:")
print(stg_production_product.head())
print("\nInformações sobre stg_production_product:")
print(stg_production_product.info())


Visualizando os primeiros registros de stg_production_product:
   productid_id             product_nm productnumber_cd  makeflag_fl  finishedgoodsflag_fl color_tp  safetystocklevel_nr  reorderpoint_tp  standardcost_vr  listprice_vr size_tp sizeunitmeasurecode_cd weightunitmeasurecode_cd  weight_tp  daystomanufacture_nr productline_tp class_tp style_tp  productsubcategoryid_id  productmodelid_id          sellstartdate_dt            sellenddate_dt                          rowguid_desc                  modifieddate_dt
0           864        CLASSIC VEST, S        VE-C304-S        False                  True     BLUE                    4                3            23.75         63.50       S                   None                     None        NaN                     0              S     None       U                        25                  1 2013-05-30 00:00:00+00:00                       NaT  EB423EF3-409D-46FE-B35B-D69970820314 2014-02-08 10:01:36.827000+00:00
1           866     

In [10]:
# Filtrar produtos relacionados a "GLOVES" no stg_production_product
gloves_products = stg_production_product[stg_production_product['product_nm'].str.contains("GLOVES", case=False, na=False)]

print("Produtos filtrados contendo 'GLOVES':")
print(gloves_products[['productid_id', 'product_nm', 'listprice_vr']].head(10))

Produtos filtrados contendo 'GLOVES':
   productid_id             product_nm  listprice_vr
4           861  FULL-FINGER GLOVES, S         37.99
5           863  FULL-FINGER GLOVES, L         37.99
6           862  FULL-FINGER GLOVES, M         37.99
7           859  HALF-FINGER GLOVES, M         24.49
8           858  HALF-FINGER GLOVES, S         24.49
9           860  HALF-FINGER GLOVES, L         24.49


In [11]:
print(gloves_products.shape)

(6, 24)


In [12]:
print("\nVisualizando os primeiros registros de fact_product_forecast:")
print(fact_product_forecast.head())
print("\nInformações sobre fact_product_forecast:")
print(fact_product_forecast.info())


Visualizando os primeiros registros de fact_product_forecast:
   store_id  productid_id forecast_date  forecast_quantity  avg_unit_price  forecast_sales_value
0         6           776    2025-04-26             400.00         2415.54             966214.44
1         3           869    2025-03-26            1256.00           52.87              66400.01
2        10           980    2025-04-26             124.00          622.68              77212.01
3         7           980    2025-04-26             100.00          622.68              62267.75
4         5           709    2025-01-26            1056.00            5.66               5974.22

Informações sobre fact_product_forecast:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106840 entries, 0 to 106839
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   store_id              106840 non-null  Int64  
 1   productid_id          106840 non-null  Int

In [13]:
# Filtrar apenas as previsões dos produtos "GLOVES"
gloves_forecast = fact_product_forecast[fact_product_forecast['productid_id'].isin(gloves_products['productid_id'])]
print("Produtos filtrados (GLOVES):")
print(gloves_forecast.head())

Produtos filtrados (GLOVES):
     store_id  productid_id forecast_date  forecast_quantity  avg_unit_price  forecast_sales_value
50          2           862    2025-04-26            1040.00           22.66              23571.46
51          1           862    2025-03-26            1252.00           22.66              28376.42
182        10           863    2025-01-26             768.00           22.32              17142.88
183         1           863    2025-03-26            2212.00           22.32              49375.07
303        10           862    2025-02-26             648.00           22.66              14686.84


In [14]:
# Converter a coluna 'forecast_date' para o tipo datetime
gloves_forecast['forecast_date'] = pd.to_datetime(gloves_forecast['forecast_date'])
print("\nDatas convertidas para datetime:")
print(gloves_forecast[['productid_id', 'forecast_date']].head())


Datas convertidas para datetime:
     productid_id forecast_date
50            862    2025-04-26
51            862    2025-03-26
182           863    2025-01-26
183           863    2025-03-26
303           862    2025-02-26


C:\Users\wrpen\AppData\Local\Temp\ipykernel_83044\3249586105.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gloves_forecast['forecast_date'] = pd.to_datetime(gloves_forecast['forecast_date'])


In [15]:
# Data mínima para identificar o ponto de partida
start_date = gloves_forecast['forecast_date'].min()
print(f"\nData mínima (início do período): {start_date}")


Data mínima (início do período): 2025-01-26 00:00:00


In [16]:
from dateutil.relativedelta import relativedelta

# Recalcular 'month_diff' com base em meses reais
gloves_forecast['month_diff'] = gloves_forecast['forecast_date'].apply(
    lambda x: relativedelta(x, start_date).months + 1
)
print("\nColuna 'month_diff' criada com meses reais:")
print(gloves_forecast[['productid_id', 'forecast_date', 'month_diff']].head())

# Criar as colunas 'month_1', 'month_2', 'month_3' com base no novo cálculo
for month in [1, 2, 3]:
    gloves_forecast[f'month_{month}'] = gloves_forecast.apply(
        lambda x: x['forecast_quantity'] if x['month_diff'] == month else 0, axis=1
    )
    print(f"\nPreenchendo os dados para 'month_{month}':")
    print(gloves_forecast[[f'month_{month}', 'forecast_quantity', 'month_diff']].head())


Coluna 'month_diff' criada com meses reais:
     productid_id forecast_date  month_diff
50            862    2025-04-26           4
51            862    2025-03-26           3
182           863    2025-01-26           1
183           863    2025-03-26           3
303           862    2025-02-26           2

Preenchendo os dados para 'month_1':
     month_1  forecast_quantity  month_diff
50      0.00            1040.00           4
51      0.00            1252.00           3
182   768.00             768.00           1
183     0.00            2212.00           3
303     0.00             648.00           2

Preenchendo os dados para 'month_2':
     month_2  forecast_quantity  month_diff
50      0.00            1040.00           4
51      0.00            1252.00           3
182     0.00             768.00           1
183     0.00            2212.00           3
303   648.00             648.00           2

Preenchendo os dados para 'month_3':
     month_3  forecast_quantity  month_diff
50   

C:\Users\wrpen\AppData\Local\Temp\ipykernel_83044\1982613389.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gloves_forecast['month_diff'] = gloves_forecast['forecast_date'].apply(
C:\Users\wrpen\AppData\Local\Temp\ipykernel_83044\1982613389.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gloves_forecast[f'month_{month}'] = gloves_forecast.apply(
C:\Users\wrpen\AppData\Local\Temp\ipykernel_83044\1982613389.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [17]:
print(gloves_forecast[gloves_forecast['month_diff'] == 3])

       store_id  productid_id forecast_date  forecast_quantity  avg_unit_price  forecast_sales_value  month_diff  month_1  month_2  month_3
51            1           862    2025-03-26            1252.00           22.66              28376.42           3     0.00     0.00  1252.00
183           1           863    2025-03-26            2212.00           22.32              49375.07           3     0.00     0.00  2212.00
590          10           862    2025-03-26             648.00           22.66              14686.84           3     0.00     0.00   648.00
591           2           862    2025-03-26            1040.00           22.66              23571.46           3     0.00     0.00  1040.00
1324          5           863    2025-03-26            1452.00           22.32              32410.76           3     0.00     0.00  1452.00
...         ...           ...           ...                ...             ...                   ...         ...      ...      ...      ...
75353         8     

In [18]:
print(gloves_forecast['forecast_date'].min(), gloves_forecast['forecast_date'].max())

2025-01-26 00:00:00 2025-04-26 00:00:00


In [19]:
# Demanda total de luvas para os próximos 3 meses
gloves_forecast['total_demand'] = gloves_forecast[['month_1', 'month_2', 'month_3']].sum(axis=1)
print("\nDemanda total calculada para cada produto (linhas de exemplo):")
print(gloves_forecast[['productid_id', 'month_1', 'month_2', 'month_3', 'total_demand']].head())


Demanda total calculada para cada produto (linhas de exemplo):
     productid_id  month_1  month_2  month_3  total_demand
50            862     0.00     0.00     0.00          0.00
51            862     0.00     0.00  1252.00       1252.00
182           863   768.00     0.00     0.00        768.00
183           863     0.00     0.00  2212.00       2212.00
303           862     0.00   648.00     0.00        648.00


C:\Users\wrpen\AppData\Local\Temp\ipykernel_83044\3432193739.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gloves_forecast['total_demand'] = gloves_forecast[['month_1', 'month_2', 'month_3']].sum(axis=1)


In [20]:
# Verificar se 'total_demand' está correto
gloves_forecast['check_total'] = gloves_forecast[['month_1', 'month_2', 'month_3']].sum(axis=1)
print("\nValidação de 'total_demand' com 'check_total':")
print(gloves_forecast[['productid_id', 'total_demand', 'check_total']].head())


Validação de 'total_demand' com 'check_total':
     productid_id  total_demand  check_total
50            862          0.00         0.00
51            862       1252.00      1252.00
182           863        768.00       768.00
183           863       2212.00      2212.00
303           862        648.00       648.00


C:\Users\wrpen\AppData\Local\Temp\ipykernel_83044\1441321352.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gloves_forecast['check_total'] = gloves_forecast[['month_1', 'month_2', 'month_3']].sum(axis=1)


In [21]:
# Somar a demanda total de todos os produtos "GLOVES"
total_gloves_demand = gloves_forecast['total_demand'].sum()
print(f"\nDemanda total de luvas nos próximos 3 meses: {total_gloves_demand}")


Demanda total de luvas nos próximos 3 meses: 2608824.0


In [22]:
# Calcular a quantidade de zíperes necessários (2 zíperes por par de luvas)
total_zippers_needed = total_gloves_demand * 2
print(f"Quantidade total de zíperes necessários: {total_zippers_needed}")

Quantidade total de zíperes necessários: 5217648.0


In [23]:
# Contexto da análise

# Justificativa:
# - O objetivo da análise foi determinar a quantidade de zíperes necessária para atender à demanda de luvas
#   nos próximos 3 meses, conforme solicitado pelo fornecedor global.
# - Considerando que cada par de luvas exige 2 zíperes, é fundamental que as previsões estejam alinhadas com
#   a demanda de luvas para evitar problemas na cadeia de suprimentos, como falta de estoque ou desperdício.

# Escopo do que foi feito:
# 1. Filtragem dos dados:
#    - Identificamos os produtos classificados como "GLOVES" no dataset `stg_production_product`.
#    - Filtramos a tabela de previsões (`dim_product_forecast`) para incluir apenas os IDs dos produtos de luvas.
# 2. Criação de colunas mensais:
#    - Calculamos as previsões de demanda para os próximos 3 meses, organizando os dados em colunas 
#      'month_1', 'month_2' e 'month_3'.
#    - Somamos essas colunas para obter a demanda total de luvas no período.
# 3. Cálculo da demanda total:
#    - Consolidamos a previsão de demanda total para todos os produtos de luvas nos próximos 3 meses.
#    - Multiplicamos a demanda total por 2 (zippers por par de luvas) para determinar a quantidade de zíperes necessária.
# 4. Validação:
#    - Verificamos se os cálculos estão consistentes ao comparar as somas das colunas mensais com a coluna de demanda total.

# Resultado:
# - Demanda total prevista de luvas: 2.608.824 pares.
# - Quantidade total de zíperes necessária: 5.217.648 unidades.